In [1]:
import time
import numpy as np

import pinocchio as pin
from pinocchio import CollisionPair, ReferenceFrame
from pinocchio.robot_wrapper import RobotWrapper

In [2]:
# URDF = "/opt/openrobots/share/example-robot-data/robots/anymal_b_simple_description/robots/anymal.urdf"
URDF = "/opt/openrobots/share/monoped_description/urdf/monoped.urdf"
robot = RobotWrapper.BuildFromURDF(URDF, root_joint=pin.JointModelFreeFlyer())

In [3]:
robot.initViewer(loadModel=True)
# floor.initViewer(loadModel=True)

In [4]:
[print(frame.name) for frame in robot.model.frames]

universe
root_joint
base
haa_joint
hipassembly
hfe_joint
upperleg
kfe_joint
lowerleg
lowerleg_contactsensor_link_joint
lowerleg_contactsensor_link


[None, None, None, None, None, None, None, None, None, None, None]

In [5]:
IDX_BASE = 2
IDX_TOOL = 10

In [6]:
# robot.collision_model.addGeometryObject(floor.collision_model.geometryObjects[0])

# robot.collision_model.addCollisionPair(CollisionPair(0, 2))
# robot.collision_model.addCollisionPair(CollisionPair(0, 3))
# robot.collision_model.addCollisionPair(CollisionPair(1, 3))

# robot.collision_model.addCollisionPair(CollisionPair(4, 5))

# robot.displayCollisions(True)
# robot.collision_data = pin.GeometryData(robot.collision_model)

In [7]:
# [print(obj.name) for obj in robot.collision_model.geometryObjects.tolist()]

In [8]:
# pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, robot.collision_data, q)
# pin.computeCollision(robot.collision_model, robot.collision_data, 0)
# pin.computeCollisions(robot.collision_model, robot.collision_data, False)

In [9]:
for k in range(len(robot.collision_model.collisionPairs)): 
    cr = robot.collision_data.collisionResults[k]
    cp = robot.collision_model.collisionPairs[k]
    print("collision pair:",cp.first,",",cp.second,"- collision:","Yes" if cr.isCollision() else "No")

In [10]:
def prox(p, mu):
    pn = np.max([p[2], 0])
    a = mu * pn
    pt = np.clip(p[:2], -a, a)
    return np.hstack([pt, pn])

In [11]:
def step(q, v):
    qm = pin.integrate(robot.model, q, dt * v / 2)
    Minv = pin.computeMinverse(robot.model, robot.data, qm)
    h = -pin.rnea(robot.model, robot.data, qm, v, aq0)
    
    ve = v + Minv @ h * dt
    
    # collision detection and resolution
    pin.framesForwardKinematics(robot.model, robot.data, qm)
    t = robot.data.oMf[IDX_TOOL].translation
    
    if t[2] < 0:
        robot.computeFrameJacobian(qm, IDX_TOOL)
        J = robot.getFrameJacobian(IDX_TOOL, ReferenceFrame.LOCAL_WORLD_ALIGNED)[:3, :]
        
        G = J @ Minv @ J.T
        c = J @ v + J @ Minv @ h * dt
        
        p = np.zeros(3)
        p[2] = pin.computeTotalMass(robot.model) * dt * 9.81
        
        r = 1 / np.max([np.linalg.det(G), 1])
        for _ in range(num_iters):
            # todo: multiple contact points
            p = prox(p - r * (G @ p + c), 1)
            
        ve += Minv @ J.T @ p
    
    q = pin.integrate(robot.model, q, dt * (v + ve) / 2)
    v = ve
    
    # correction
    if t[2] < 0:
        pass
    
    if False:
        g = np.array([0, 0, -t[2]])
        robot.computeFrameJacobian(q, IDX_TOOL)
        J = robot.getFrameJacobian(IDX_TOOL, ReferenceFrame.LOCAL_WORLD_ALIGNED)[:3, :]
        q = pin.integrate(robot.model, q, np.linalg.pinv(J) @ g)
        
    return q, v

In [12]:
q0 = robot.q0
v0 = robot.v0

q0[2] = 1
v0[0] = 0

aq0 = np.zeros(robot.nv)

In [13]:
dt = 0.01
slow = 10
num_iters = 100

In [14]:
q = q0
v = v0

# v[6] = 10
# v[7] = 10

In [15]:
while True:
    start_time = time.time()
    
    q, v = step(q, v)
    robot.display(q)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    sleep_time = max([dt * slow - elapsed_time, 0])
    time.sleep(sleep_time)

KeyboardInterrupt: 